Произведем небольшую практику по принименению библиотеки Пандас 

Для датафрейма log с данными о переходах на сайт из создадим столбец source_type по правилам:

- если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
- для источников paid и email из России ставим ad;
- для источников paid и email не из России ставим other;
- все остальные варианты берём из traffic_source без изменений.

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv('visit_log.csv', sep = ';')
df.loc[10:15]

,timestamp,visit_id,url,region,user_id,traffic_source
10,1549980777,01d448afd9,https://host.ru/a5dda93e70318570c0,India,27e76a4a80,direct
11,1549980785,71b6c1d538,https://host.ru/c8d9213a31839f9a3a,Ukraine,3b13701488,google
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid
13,1549980792,dd46c3eebb,https://host.ru/3004a8273caeef2867,China,7e301c8c41,direct
14,1549980793,e7ecebbc59,https://host.ru/05e41944dd85a900a6,Belarus,ac52dedc3b,direct
15,1549980797,5322fecfc9,https://host.ru/4adc8a29b212051b04,Belarus,9a45d98867,paid


In [ ]:
df['source_type'] = (df.traffic_source)
df.loc[(df.traffic_source == 'yandex')|(df.traffic_source == 'google'), 'source_type'] = 'organic'
df.loc[(df.traffic_source == 'paid')|(df.traffic_source == 'email')&(df.region == 'Russia'),'source_type'] = 'ad'
df.loc[(df.traffic_source == 'paid')|(df.traffic_source == 'email')&(df.region != 'Russia'), 'source_type'] = 'other'

df = df.fillna(df.traffic_source)

df.loc[10:15]

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
10,1549980777,01d448afd9,https://host.ru/a5dda93e70318570c0,India,27e76a4a80,direct,direct
11,1549980785,71b6c1d538,https://host.ru/c8d9213a31839f9a3a,Ukraine,3b13701488,google,organic
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,other
13,1549980792,dd46c3eebb,https://host.ru/3004a8273caeef2867,China,7e301c8c41,direct,direct
14,1549980793,e7ecebbc59,https://host.ru/05e41944dd85a900a6,Belarus,ac52dedc3b,direct,direct
15,1549980797,5322fecfc9,https://host.ru/4adc8a29b212051b04,Belarus,9a45d98867,paid,other


В файле URLs.txt содержатся URL страниц новостного сайта. Отфильтруем его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис.

In [ ]:
df = pd.read_csv('URLs.txt')
df.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
df = df[df["url"].str.contains(r"/\d{8}-")]
df.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


Посчитаем среднее время жизни пользователей, которые выставили более 100 оценок из файла с данными об оценках пользователя

In [ ]:
df = pd.read_csv('ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
live_tmp = df.groupby('userId')['timestamp']\
        .agg({'count',  lambda x: max(x) - min(x)}).reset_index()\
        .rename(columns={'<lambda_0>':'lt'})
        
live_tmp[live_tmp['count'] > 100]['lt'].mean()

40080507.4496124

Дана статистика услуг по перевозкам клиентов по типам: ЖД, Авто и авиа - перевозки.

Сформируем две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
- аналогичную таблицу по типам выручки с указанием адреса клиента.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
rzd.merge(auto, how = 'outer', on = 'client_id')\
    .merge(air, how = 'outer', on = 'client_id')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [ ]:
client_base\
    .merge(auto, how = 'left', on = 'client_id')\
    .merge(air, how = 'left', on = 'client_id')\
    .merge(rzd, how = 'left', on = 'client_id')

,client_id,address,auto_revenue,air_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,57483.0,NaN,10283.0
3,114,Мира 14,83.0,NaN,5774.0
4,115,ЗЖБИиДК 1,912.0,81.0,981.0
5,116,Строителей 18,4834.0,4.0,NaN
6,117,Панфиловская 33,98.0,13.0,NaN
7,118,Мастеркова 4,NaN,173.0,NaN
